# 7.2.2. Multi Layer Perception Model

1. See Notion Notes for short-hand notation, explanations, my thoughts, etc.
2. BOOKS:
    - Deep Learning for Time Series Forecasting - Predict the Future with MLPs, CNNs and LSTMs in Python by Jason Brownlee
    - Introduction to TSF with Python - How to Prepare Data and Develop Models to Predict the Future by Jason Brownlee

# Imports + Load Data

In [1]:
import numpy as np
import pandas as pd


from keras.layers import Dense
from keras.models import Sequential
from sklearn.metrics import mean_squared_error

2023-05-25 17:42:41.304864: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
BASE = '/Users/brinkley97/Documents/development/'

In [3]:
PATH_TO_BOOK = 'book-intro_to_tsf_with_python/'
TS_AS_SML_FILE = BASE + PATH_TO_BOOK + '3-timeSeriesAsSupervisedLearning.ipynb'
TS_AS_SML_FILE

'/Users/brinkley97/Documents/development/book-intro_to_tsf_with_python/3-timeSeriesAsSupervisedLearning.ipynb'

In [4]:
# %load TS_AS_SML_FILE

In [5]:
# %run $TS_AS_SML_FILE

# For Full Datasets

In [6]:
dataset = BASE + PATH_TO_BOOK + 'datasets/daily-min-temperatures.csv'
series = pd.read_csv(dataset, header=0, index_col=0)
# series

In [7]:
data = series.values
data

array([[20.7],
       [17.9],
       [18.8],
       ...,
       [13.5],
       [15.7],
       [13. ]])

In [8]:
# split a univariate dataset into train/test sets 
def train_test_split(data, n_test):
    print(len(data[:-n_test]))
    print(len(data[-n_test:]))
    return data[:-n_test], data[-n_test:]

In [9]:
# def split_data(specific_features):
#     """Split data into TRAIN, TEST, and VALIDATION sets
    
#     Parameters:
#     specific_features -- list (of the reduced features)
#     specific_features_true_labels -- list (of the emotion labels that belong to that specifuc feature)
#     test_size -- float (to pass into sklearn train_test_split())
    
#     Return:
#     X_train, X_test, X_val, y_train, y_test, y_val -- list (for that specific subset of the features)
    
#     """
#     print(specific_features)
#     X_train, X_test, y_train, y_test = train_test_split(specific_features, test_size=0.2, random_state=1)
#     total_X = len(X_train) + len(X_test)
#     total_Y = len(y_train) + len(y_test)
    
#     print("[INFO] X, y TRAIN sets")
#     print(np.shape(X_train), np.shape(y_train))
    
#     print("\n[INFO] X, y TEST sets")
#     print(np.shape(X_test), np.shape(y_test))
#     # print("[INFO] TOTAL TRAIN, TEST sets")
#     # print(total_X, total_Y)
    
#     return X_train, X_test, y_train, y_test

In [10]:
# data split 
n_test = 12 
training_data, test_data = train_test_split(data, n_test)

3638
12


In [11]:
# training_data

In [12]:
def convert_uts_sequence_to_sml(uts_observations, prior_observations, forecasting_step):
    """Splits a given UTS into multiple input rows where each input row has a specified number of timestamps and the output is a single timestamp.
    
    Parameters:
    uts_observations -- 1D np array (of UTS data to transform to SML data with size  b rows/length x 1 dimension)
    prior_observations -- py int (of all observations before we get to where we want to start making the predictions)
    forecasting_step -- py int (of how far out to forecast, 1 only the next timestamp, 2 the next two timestamps, ... n the next n timestamps)
    
    Return:
    agg.values -- np array (of new sml data)
    """
    
    df = pd.DataFrame(uts_observations)
    cols = list()
    
    # input sequence (t-n, ... t-1)
    for prior_observation in range(prior_observations, 0, -1):
        cols.append(df.shift(prior_observation))
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, forecasting_step):
        cols.append(df.shift(-i))
        
        # put it all together
        agg = pd.concat(cols, axis=1) 
        
        # drop rows with NaN values
        agg.dropna(inplace=True)
    # print(agg)
    
    return agg.values

In [13]:
prior_observations, forecasting_step = [24, 1]
converted_uts_to_sml = convert_uts_sequence_to_sml(training_data, prior_observations, forecasting_step) 
converted_uts_to_sml

array([[20.7, 17.9, 18.8, ..., 14.4, 16. , 16.5],
       [17.9, 18.8, 14.6, ..., 16. , 16.5, 18.7],
       [18.8, 14.6, 15.8, ..., 16.5, 18.7, 19.4],
       ...,
       [10.5, 11.1, 13. , ..., 13.4, 13.6, 13.9],
       [11.1, 13. , 12.9, ..., 13.6, 13.9, 17.2],
       [13. , 12.9,  8.8, ..., 13.9, 17.2, 14.7]])

In [14]:
# fit a model 
def mlp_model_fit(sml_data, n_nodes, n_epochs, n_batch, prior_observations): 
    """
    Parameters: 
    sml_data -- 
    
    """
    
    # prepare data 
    # converted_uts_to_sml = convert_uts_sequence_to_sml(train, prior_observations, forecasting_step) 
    print(sml_data)
    
    train_x, train_y = sml_data[:, :-1], sml_data[:, -1] 
    # define model
    model = Sequential() 
    model.add(Dense(n_nodes, activation='relu' , input_dim=prior_observations)) 
    model.add(Dense(1)) 
    model.compile(loss='mse' , optimizer='adam') 
    
    # fit
    model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0) 
    
    return model

In [15]:
n_nodes, n_epochs, n_batch = [500, 100, 100] 
print(n_epochs)
model = mlp_model_fit(converted_uts_to_sml, n_nodes, n_epochs, n_batch, prior_observations) 

100
[[20.7 17.9 18.8 ... 14.4 16.  16.5]
 [17.9 18.8 14.6 ... 16.  16.5 18.7]
 [18.8 14.6 15.8 ... 16.5 18.7 19.4]
 ...
 [10.5 11.1 13.  ... 13.4 13.6 13.9]
 [11.1 13.  12.9 ... 13.6 13.9 17.2]
 [13.  12.9  8.8 ... 13.9 17.2 14.7]]


2023-05-25 17:42:44.060367: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
model

In [17]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return np.sqrt(mean_squared_error(actual, predicted))

In [18]:
# forecast with a pre-fit model 
def model_predict(model, history, n_input): 
    # unpack config 
    # n_input, _, _, _ = config 
    # prepare data 
    x_input = np.array(history[-n_input:]).reshape(1, n_input) 
    # forecast 
    yhat = model.predict(x_input, verbose=0) 
    
    return yhat[0]

In [21]:
# walk-forward validation for univariate data 
def walk_forward_validation(training_data, n_test, model, cfg, test_data): 
    predictions = list()
    
    # split dataset 
    # train, test = train_test_split(data, n_test) 
    # fit model 
    # model = model_fit(train, cfg) 
    # seed history with training dataset 
    history = [x for x in training_data]
    
    # step over each time-step in the test set 
    for i in range(len(test_data)): 
        # fit model and make forecast for history 
        yhat = model_predict(model, history, cfg) 
        
        # store forecast in list of predictions 
        predictions.append(yhat) 
        
        # add actual observation to history for the next loop 
        history.append(test_data[i])
        
    # estimate prediction error 
    error = measure_rmse(test_data, predictions) 
    print('> %.3f' % error) 
    
    return error

In [22]:
walk_forward_validation(training_data, n_test, model, prior_observations, test_data)

> 1.672


1.6715478050695394

In [33]:
# repeat evaluation of a config 
def repeat_evaluate(training_data, n_test, model, prior_observations, n_repeats=30): 
    
    # fit and evaluate the model n times 
    scores = [walk_forward_validation(training_data, n_test, model, prior_observations, test_data) for _ in range(n_repeats)]
    
    return scores

In [34]:
# summarize model performance
def summarize_scores(name, scores): 
    # print a summary
    scores_m, score_std = mean(scores), std(scores)
    print(' %s: %.3f RMSE (+/- %.3f)' % (name, scores_m, score_std)) 
    
    # box and whisker plot 
    pyplot.boxplot(scores)
    pyplot.show()

In [35]:
# train, test = split_data(data)

In [36]:
# config = [24, 500, 100, 100] 
# model = model_fit(train, cfg) 

In [37]:
# # define config
# config = [24, 500, 100, 100] 

# # grid search 
# scores = repeat_evaluate(data, config, n_test)
scores = repeat_evaluate(training_data, n_test, model, prior_observations, test_data)

TypeError: only integer scalar arrays can be converted to a scalar index

In [55]:
# summarize scores 
summarize_scores('mlp' , scores)

NameError: name 'scores' is not defined